In [20]:
from pathlib import Path
from llama_index import download_loader
import dotenv
import os
from llama_index import VectorStoreIndex

env_path = Path('./RAG/.env')

if env_path.exists():
    dotenv.load_dotenv(env_path)
else:
    print("No .env file found")


True

In [4]:
MarkdownReader = download_loader("MarkdownReader")

loader = MarkdownReader()


In [6]:
path = os.getenv("PATH_DOC")
documents = loader.load_data(file=Path(path))

In [6]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(documents)

In [12]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")


In [13]:
from llama_index import (
    VectorStoreIndex,
    get_response_synthesizer,
)
#build index
index = VectorStoreIndex.from_documents(documents, show_progress=True)

Generating embeddings: 100%|██████████| 26/26 [00:03<00:00,  8.10it/s]


In [ ]:
from llama_index import (
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor


# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.7)
    ]

)



In [17]:
# query
response = query_engine.query("Can I be arrested?")
print(response)
response.source_nodes



No, you cannot be arrested for not linking your PAN and Aadhaar cards. However, you may be subject to a penalty charge for the delay in linking them.


[NodeWithScore(node=TextNode(id_='925e0b04-1383-419a-9e2d-afad94e6ccd7', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='8d64f6e1-dcfd-4f58-8ded-497d7def8ffa', node_type=None, metadata={}, hash='565f0202b24effe65ea76890d2bcda94578e5c98204c2f650557620fa868821e')}, hash='57264f2456b075534bd4609de42ba114a1f9b8120aa30db7135014e842f27849', text='The charges for linking Pan & Aadhaar is INR 2000/-, including the penalty charges to be paid for the delay in linking Pan & Aadhaar.', start_char_idx=0, end_char_idx=133, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.7272593637622214),
 NodeWithScore(node=TextNode(id_='2c56c937-eabb-4741-9d3f-f265a49ef89c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_